In [ ]:
from BinaryFileUnpack import BinaryFileUnpack
import numpy as np
import tkinter as tk
from tkinter import filedialog
from matplotlib import pyplot as plt

# Kistler Lab Data Instrutions with <code>BinaryFileUnpack</code>
This Jupyter Notebook contains an example of the analysis methods in the <code>BinaryFileUnpack</code> module.

**Instructions**
1. Under the "Daily Calibration" cell, indicate which file should be used for the calibration of pressure data, along with the ambient pressure, indicated in the filename. The calibrated pressure data is stored in a new variable named <code>Pressure</code>, while the original pressure is in <code>analysis.P</code>.
    - If there is no such file, ignore the "Daily Calibration" and "Applying Calibration" cells. When plotting, replace variables named <code>Pressure</code> with <code>analysis.P</code>.
2. Select the <code>.bin</code> file corresponding to the experiment to analyze.
3. If there is a daily calibration, run the cell labeled "Apply Calibration."
4. Run whatever cell you desire!


When it comes to plotting, there are 2 options:
- <code>%matplotlib qt</code>: A Jupyter Notebook magic command that emulates running <code>Matplotlib</code> plots in a <code>.py</code> file. This option will open up a separate, interactive window for the graph, allowing you to select a range of data more flexibly.
- <code>%matplotlib inline</code>: A Jupyter Notebook magic command that displays plots within the notebook. While less flexible, it can handle plots containing more data better than the <code>%matplotlib qt</code> option.

Choose which option you want by commenting out the option you do not want!


## Daily Calibration

In [ ]:
def daily_cali(fp:str, atm_pres):
    cali_file = BinaryFileUnpack(fp)
    pres_cali = np.mean(cali_file.P, axis=-1) - atm_pres
    return pres_cali

def apply_cali(analysis:BinaryFileUnpack, pres_cali):
    Pres = np.zeros(analysis.P.shape)
    for i in range(analysis.num_sens):
        Pres[i] = analysis.P[i] - pres_cali[i]
    return Pres

pres_cali_1025 = daily_cali(r"C:\Users\akyap\OneDrive\Documents\Academics\Research\LDEO Geysers\files\10-25\calibration_1019p01-20241025-10-14-35.bin", 1.01901)
pres_cali_1025

## Initializing BinaryFileUnpack object
Select the <code>.bin</code> file to analyze.

In [ ]:
# Select the file to analyze
interactive_select = True

if interactive_select:
    root = tk.Tk()
    root.withdraw()
    filepath = filedialog.askopenfilename()
else:
    # Specify filepath
    filepath = r'HotWaterCycles_heaters_on_steam_in-20221118-20-09-42.bin'
print(filepath)

In [ ]:
analysis = BinaryFileUnpack(filepath, cut_range=None, sens_from_header=True, new_file=False)
num_sens = analysis.num_sens
analysis.header_info

### Applying Calibration

In [ ]:
Pressure = apply_cali(analysis, pres_cali_1025)
Pressure

## Graphing PT data in Matplotlib (static)

In [ ]:
# %matplotlib qt
%matplotlib inline

sta = 0; end = len(analysis.time)/analysis.fs
times = analysis.getTimeRange(sta, end)
# Pressure
analysis.plot_static(analysis.time, Pressure, "time (s)", "Pressure (bar)", plots_shape=(num_sens, 1), color='b', times=times, ordering=[3, 5, 4, 2, 1, 0])
# Temperature
analysis.plot_static(analysis.time, analysis.T, "time (s)", r"Temperature ($^{\circ}C$)", plots_shape=(num_sens, 1), color='r', times=times, ordering=[3, 5, 4, 2, 1, 0])
# PT
analysis.plot_static(analysis.time, analysis.P, "time (s)", "Pressure (bar)", plots_shape=(num_sens, 1), times=times, y2 = analysis.T, ordering=[3, 5, 4, 2, 1, 0], y2_label=r"Temperature ($^{\circ}C$)", sharex=True)

Pressure on same axis, Temperature on same axis of all sensors

In [ ]:
# %matplotlib qt
%matplotlib inline
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].set_xlabel("Time (sec)")
ax[0].set_ylabel("Pressure (bar)")
ax[1].set_ylabel(r"Temperature ($^{\circ}C$)")
for i in range(analysis.num_sens):
    ax[0].plot(analysis.time, Pressure[i], label=f"Sensor {i+1}", linewidth=0.5)
for i in range(analysis.num_sens):
    ax[1].plot(analysis.time, analysis.T[i], label=f"Sensor {i+1}", linewidth=0.5)
ax[0].legend(bbox_to_anchor=(1.02, 0), loc='lower left', borderaxespad=0.)
ax[1].legend(bbox_to_anchor=(1.02, 0), loc='lower left', borderaxespad=0.)
fig.show()

Same as above, but just with filtered Pressure data.

In [ ]:
# %matplotlib qt
%matplotlib inline
import scipy.signal as signal
bound_freq = 2.2
nyq_freq = analysis.fs // 2
b, a = signal.butter(1, bound_freq/nyq_freq, 'lowpass')
for i in range(analysis.num_sens):
    plt.plot(analysis.time, signal.filtfilt(b, a, Pressure[i]), label=f"Sensor {i+1}", linewidth=0.5)
plt.legend()
plt.xlabel("Time (sec)")
plt.ylabel("Pressure (bar)")
plt.show()

## Simple Spectral Analysis
For a more involved spectral analysis, run the "Spectogram cells." This gives the spectra of the entire data.

Obtaining spectra

In [ ]:
# Pressure
Pxx = analysis.spectra(Pressure)
# Temperature
Txx = analysis.spectra(analysis.T)

Plotting Spectra (buggy)

In [ ]:
# Spectral Analysis
# %matplotlib qt
%matplotlib inline
analysis.plot_static(Pxx[0], Pxx[1], "Frequency (Hz)", "Power (dB)", plots_shape=(num_sens//2, 2), color='b', x_axis_type='log')
analysis.plot_static(Txx[0], Txx[1], "Frequency (Hz)", "Power (dB)", plots_shape=(num_sens//2, 2), color='r', x_axis_type='log')

## Spectrogram

For a more detailed analysis, should run this. 

Obtaining spectogram

In [ ]:
import scipy.fft as fft
# FFT on Data
# Pick a sensor
sen:int = 1
data = Pressure[sen]
spec = analysis.spectrogram(data, window=20)

Plotting Spectrum

In [ ]:
# %matplotlib qt
%matplotlib inline

fig, ax = plt.subplots(2, 1, height_ratios=[1, 5], sharex=False)
ax[0].plot(analysis.time[:-1], analysis.P[sen, :-1], color='blue', linewidth=0.5)
ax[0].set_ylabel('Pressure (bar)')
ax[0].set_xlim((0, analysis.time[-1]))

ax[1].set_xlabel('Time (sec)')
ax[1].set_ylabel('Frequency (Hz)')
ax[1].pcolormesh(np.arange(int(analysis.time[-1])+1), spec[0, :, 0], spec[:, :-1, 1].T, cmap='viridis_r', vmin=-5, vmax=10, shading='flat')
ax[1].set_ylim(0, 500)

fig.show()

## Plotting Pressure against Temperature Data for Hot Water Eruption Cycles

In [ ]:
# %matplotlib qt
%matplotlib inline
analysis.plot_eruption_PT(show_phase_boundaries=True, title=f"10-25 Eruption Stage 4", ordering=[3, 5, 4, 2, 1, 0], savefig=False, times=times)

In [ ]:
header = "Time,Pressure (bar),Temperature (deg C),Sensor No.\n"
csv_name = "TwoConstrictions_TopMiddle_Pool_Stage2and3-20241004-15-07-36-data.csv"
with open(csv_name, 'w') as fw:
    for i in range(analysis.P.shape[0]):
        for j in range(analysis.P.shape[1]):
            fw.write(f"{times[j]},{analysis.P[i][j]},{analysis.T[i][j]},{int(i+1)}\n")

In [ ]:
x = np.array([0,1,2,3,4,5,6,7,8,9])
x[:3]

In [ ]:
try:
    with open(r"..\11-08\calibration_Room_994p23-20241108-10-34-34_headerInfo.csv", 'r') as fr:
        headers = fr.readlines()
    with open('new.csv', 'w') as fw:
        fw.writelines(headers)
        fw.write(f"Start Splice,120\n")
except:
    pass